In [65]:
import os
from natsort import os_sorted
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [66]:
rootdir = 'outputs'
dir_list = []
#list the models
for subdir, dirs, files in os.walk(rootdir):
    for dirname in os_sorted(dirs):
        dir_list.append(rootdir+os.sep+dirname)
print(dir_list)

['outputs\\qn_multistate-non-unique', 'outputs\\qn_singlestate-non-unique', 'outputs\\qn_singlestate-unique']


In [79]:
def getData(dir, headers, amount=30):
    i = 1
    df = pd.DataFrame(columns=headers)
    for subdir, dirs, files in os.walk(dir):
        for filename in files:
            filepath = subdir + os.sep + filename
            if filepath.endswith(".csv"):
                if i == amount+1:
                    return df
                df_temp = pd.read_csv(filepath, skiprows = 1, names= headers)
                run = [i]*len(df_temp)
                df_temp['run'] = run
                
                df = df.merge(df_temp, how='outer', on=headers)
                i += 1
    return df

In [80]:
headers = ['episode','step','episode_reward','epsilon','x_position','y_position', 'distance', 'run']
dfMultiNonUnique = getData('outputs\\qn_multistate-non-unique', headers)
dfMultiUnique = getData('outputs\\qn_multistate-unique', headers)
dfSingleNonUnique = getData('outputs\\qn_singlestate-non-unique', headers)
dfSingleUnique = getData('outputs\\qn_singlestate-unique', headers)



### Put all functions here

In [77]:
def getRewardPerEpisode(df):
    x=[]
    y=[]
    for run in range(1, df['run'].max()+1):
        df_temp = df[df['run'] == run]
        for episode in range(1,301):
            x.append(episode)
            y.append(df_temp['episode_reward'][df['episode'] == episode].max())
    return x,y

def getSteps(df):
    x=[]
    y=[]
    for run in range(1, df['run'].max()+1):
        df_temp = df[df['run'] == run]
        for episode in range(1,301):
            x.append(episode)
            y.append(df_temp['steps'][df['episode'] == episode].max())
            

,episode,step,episode_reward,epsilon,x_position,y_position,distance,run
0,1,1,-0.5,1.000000,55,12,59,1
1,1,2,-4.0,0.990000,56,11,61,1
2,1,3,-6.0,0.980100,57,10,62,1
3,1,4,-5.0,0.970299,56,10,61,1
4,1,5,-5.5,0.960596,55,9,61,1
...,...,...,...,...,...,...,...,...
27917731,300,68,45.5,0.049536,12,19,6,100
27917732,300,69,46.5,0.049536,11,19,5,100
27917733,300,70,47.5,0.049536,10,18,4,100
27917734,300,71,48.5,0.049536,9,19,3,100


In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
ax.set_xlabel('Episode')
ax.set_ylabel('Rewards')
ax.set_title('Rewards per Episode')

x,y = getRewardPerEpisode(dfMultiNonUnique)
sns.lineplot(x=x,y=y, label='Multi - Non Unique')

x,y = getRewardPerEpisode(dfSingleNonUnique)
sns.lineplot(x=x,y=y, label='Single - Non Unique')

x,y = getRewardPerEpisode(dfMultiUnique)
sns.lineplot(x=x,y=y, dashes = True, label='Multi - Unique')

x,y = getRewardPerEpisode(dfSingleUnique)
sns.lineplot(x=x,y=y,dashes = True, label='Single - Unique')

In [ ]:
fig.savefig('rewards.png')